<h1><center>Capstone Project: Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h3><center>Get Latitude & Longitude</center></h3>
<h4>About this Notebook</h4>
<br>created by <a href="https://github.com/Sherryl93/">@Sherryl93</a>

<br>
<br> Data Source: <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>

In [2]:
#Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# fetch data from wikipedia and parse the HTML/XML 
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
# extracting the raw table inside that webpage
table = soup.find('table')

## Extract data from Raw Table

In [4]:
#Declare empty list
postal_list   = []
borough_list  = []
neighbor_list = []


# begin loop to fetch data 
for tr_cell in table.find_all('tr'):
    for td_cell in tr_cell.find_all('td'):
        #print(td_cell)
        # loop trhough postal:
        for b in td_cell.find_all('b'):
            postal_list.append(b.string)
        if not(td_cell.find_all('i') or td_cell.find_all('a')):
            str1,str2 = td_cell.find('span',{'style':'font-size:80%;'}).text.replace(')', '').split('(')
            borough_list.append(str1)
            neighbor_list.append(str2)
        elif td_cell.find_all('i'):
            for i in td_cell.find_all('i'):
                borough_list.append(i.string)
            neighbor_list.append(None)
        else:
            counter = 0
            temp_neigh  = [] #for temp only
            for a in td_cell.find_all('a'):
                #print(a.get('title'))
                if counter == 0:
                    borough_list.append(a.string)
                    #print(a.string)
                    counter+=1
                else:
                    temp_neigh.append(a.string)
            if len(temp_neigh)>1:
                neighbor_list.append(','.join(temp_neigh))
            else:
                try:
                    neighbor_list.append(temp_neigh[0])
                except:
                    neighbor_list.append(None)
                    
# integrate
data = {'PostalCode': postal_list, 'Borough': borough_list, 'Neighborhood':neighbor_list}
df = pd.DataFrame.from_dict(data)

#Drop column if column contained not assigned
df_cleaned = df[(df.Borough != 'Not assigned')& (df.Borough.notna()) & (df.Neighborhood.notna())]
df_cleaned.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [11]:
 #geo df
url="http://cocl.us/Geospatial_data"
geo_df=pd.read_csv(url)

In [12]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
geo_df = geo_df.rename(columns={"Postal Code": "PostalCode"})

In [15]:
df_geo = pd.merge(df_cleaned, geo_df, on='PostalCode')
df_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937
